## KEARS
Keras is a powerful easy to use Python library for developing and evaluating deep learning models.<br>
Implemented simple keras model <br>

In [1]:
from keras.models import Sequential
from keras.layers import Dense
import numpy as np
import csv
from skimage.transform import resize
from skimage.io import imread, imsave
import warnings
warnings.filterwarnings('ignore');

Using TensorFlow backend.


### Load the data
traing dataset and testing dataset is loaded along wiht the label data file<br>
no of rows, cols, channel(rgb) is initiatization<br>
here will resize the images


In [2]:
def loadDataAndLabels(myDataDir):
    imgFiles = []
    myLabel = []
    with open(myDataDir+'/aa-labels.csv','r') as fi:
        reader =  csv.reader(fi)
        ignoreHead = 0
        for row in reader:
            #print(row)
            if len(row) > 0:
                if ignoreHead == 0:
                    ignoreHead = 1
                else:
                    imgFiles.append(row[0])
                    myLabel.append(row[1])
    fi.close()
    num_ims = len(imgFiles)
    num_rows = 100
    num_cols = 100
    num_chans = 3
    resize_factor = 4
    num_rows_resize = int(num_rows/resize_factor)
    num_cols_resize = int(num_cols/resize_factor)
    myDataNp = np.zeros((num_ims,num_rows_resize,num_cols_resize,num_chans))
    myLabelNp = np.array(myLabel)
    i = 0
    for imgName in imgFiles:
        img_data = resize(imread(myDataDir+"/"+imgName+".jpg"),(num_rows_resize,num_cols_resize),mode='constant')
        myDataNp[i,:,:,:] = img_data[:,:,:]
        if i % 1000 == 0:
            print(i,": loaded img - ", myDataDir,"/",imgName,".jpg")
        i = i+1
    myDataNp = myDataNp.reshape(num_ims,num_rows_resize*num_cols_resize*num_chans)
    return(myDataNp,myLabelNp)

In [3]:
training_data = 'C:/Users/sonali/Desktop/DS/ml/project_final/fruits/fruits-360/modTraining_medtraj1'
# Load the training set
(X_train,y_train) = loadDataAndLabels(training_data)

0 : loaded img -  C:/Users/sonali/Desktop/DS/ml/project_final/fruits/fruits-360/modTraining_medtraj1 / Apple Braeburn_0_100_f0 .jpg
1000 : loaded img -  C:/Users/sonali/Desktop/DS/ml/project_final/fruits/fruits-360/modTraining_medtraj1 / Apple Golden 2_107_100_f7 .jpg
2000 : loaded img -  C:/Users/sonali/Desktop/DS/ml/project_final/fruits/fruits-360/modTraining_medtraj1 / Apple Granny Smith_113_100_f1 .jpg
3000 : loaded img -  C:/Users/sonali/Desktop/DS/ml/project_final/fruits/fruits-360/modTraining_medtraj1 / Apple Red 2_117_100_f3 .jpg
4000 : loaded img -  C:/Users/sonali/Desktop/DS/ml/project_final/fruits/fruits-360/modTraining_medtraj1 / Apple Red Delicious_125_100_f2 .jpg
5000 : loaded img -  C:/Users/sonali/Desktop/DS/ml/project_final/fruits/fruits-360/modTraining_medtraj1 / Apple Red Yellow 2_156_100_f7 .jpg
6000 : loaded img -  C:/Users/sonali/Desktop/DS/ml/project_final/fruits/fruits-360/modTraining_medtraj1 / Avocado_136_100_f2 .jpg
7000 : loaded img -  C:/Users/sonali/Deskto

In [4]:
testing_data = 'C:/Users/sonali/Desktop/DS/ml/project_final/fruits/fruits-360/modTest_medtraj1'
# Load the test set
(X_test,y_test) = loadDataAndLabels(testing_data)

0 : loaded img -  C:/Users/sonali/Desktop/DS/ml/project_final/fruits/fruits-360/modTest_medtraj1 / Apple Braeburn_321_100_f0 .jpg
1000 : loaded img -  C:/Users/sonali/Desktop/DS/ml/project_final/fruits/fruits-360/modTest_medtraj1 / Apple Red Yellow 2_112_100_f4 .jpg
2000 : loaded img -  C:/Users/sonali/Desktop/DS/ml/project_final/fruits/fruits-360/modTest_medtraj1 / Carambula_146_100_f8 .jpg
3000 : loaded img -  C:/Users/sonali/Desktop/DS/ml/project_final/fruits/fruits-360/modTest_medtraj1 / Granadilla_105_100_f0 .jpg
4000 : loaded img -  C:/Users/sonali/Desktop/DS/ml/project_final/fruits/fruits-360/modTest_medtraj1 / Huckleberry_100_100_f11 .jpg
5000 : loaded img -  C:/Users/sonali/Desktop/DS/ml/project_final/fruits/fruits-360/modTest_medtraj1 / Mangostan_132_100_f6 .jpg
6000 : loaded img -  C:/Users/sonali/Desktop/DS/ml/project_final/fruits/fruits-360/modTest_medtraj1 / Pear_322_100_f1 .jpg
7000 : loaded img -  C:/Users/sonali/Desktop/DS/ml/project_final/fruits/fruits-360/modTest_med

check the no of rows and columns in training and testing datasets which is used to inpudt values to the keras layer

In [5]:
#shape of the file
print('training datasets:\n Rows: %d,  Columns: %d' %(X_train.shape[0], X_train.shape[1]))
print('testing datasets:\n Rows: %d,  Columns: %d' %(X_test.shape[0], X_test.shape[1]))

training datasets:
 Rows: 44994,  Columns: 1875
testing datasets:
 Rows: 9053,  Columns: 1875


In [6]:
#avy and std of the images in training and testing dataset
mean_vals = np.mean(X_train, axis=0)
std_val = np.std(X_train)

X_train = (X_train - mean_vals)/std_val
X_test = (X_test - mean_vals)/std_val
  
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(44994, 1875) (44994,)
(9053, 1875) (9053,)


### Define the model
create a Sequential model and add layers, set the  input parameter to 1875 as per the no of cols.<br>
Fully connected layers are defined using the Dense class. 
* first argument will be no of neurons in the layer
* the initialization method as the second argument as init and the activation function 
* network weights are in between 0 and 0.05 because that is the default uniform weight initialization in Keras. 
* relu activation function on the first two layers and the sigmoid function in the output layer.
* The first layer has 100 neurons and expects 1875 input variables. 
* The second hidden layer has 50 neurons  and third 20 neurons
* the output layer has 1 neuron for predication

In [33]:
np.random.seed(7)
# create model
model = Sequential()
model.add(Dense(100, input_dim=1875, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(20, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

### complie the model
* here We must specify the loss function to use to evaluate a set of weights, 
* the optimizer is used to search through different weights for the network
* I have used loss as binary_crossentropy. 
* optimizer as adam


In [34]:
# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

### fit the model
her we need difinf the no od epochs and batch_size

In [31]:
# Fit the model
model.fit(X_train, y_train, epochs=20, batch_size=10)

Epoch 1/20
44994/44994 [==============================] - 17s 370us/step - loss: -12.5139 - acc: 0.3850
Epoch 2/20
44994/44994 [==============================] - 16s 347us/step - loss: -13.0558 - acc: 0.4171
Epoch 3/20
44994/44994 [==============================] - 13s 289us/step - loss: -13.1910 - acc: 0.4255
Epoch 4/20
44994/44994 [==============================] - 14s 312us/step - loss: -13.2709 - acc: 0.4317
Epoch 5/20
44994/44994 [==============================] - 15s 343us/step - loss: -13.3220 - acc: 0.4351
Epoch 6/20
44994/44994 [==============================] - 16s 354us/step - loss: -13.3730 - acc: 0.4376
Epoch 7/20
44994/44994 [==============================] - 16s 362us/step - loss: -13.4084 - acc: 0.4403
Epoch 8/20
44994/44994 [==============================] - 16s 359us/step - loss: -13.4294 - acc: 0.4401
Epoch 9/20
44994/44994 [==============================] - 16s 356us/step - loss: -13.4564 - acc: 0.4425
Epoch 10/20
44994/44994 [==============================] - 16s 3

### model evaluation
generate the  prediction for each input and output pair and collect scores, 
including the average loss and  accuracy.

In [32]:
# evaluate the model
scores = model.evaluate(X_train, y_train)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

44994/44994 [==============================] - 2s 41us/step

acc: 45.30%
